In [1]:
import tensorflow as tf
from tensorflow.keras import datasets,layers,models,optimizers
import datetime, os

2024-03-24 08:15:59.972287: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-24 08:15:59.972421: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-24 08:16:00.128857: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
EPOCHS = 5
BATCH_SIZE = 128
VERBOSE = 1
OPTIMIZER = tf.keras.optimizers.Adam()
VALIDATION_SPLIT = 0.95
IMG_ROWS,IMG_COLS = 28,28
#input image dimensions
INPUT_SHAPE = (IMG_ROWS,IMG_COLS,1)
NB_CLASSES = 10 # number of outputs = number of digits

In [5]:
#define the convnet
def build(input_shape,classes):
    model = models.Sequential()
    # CONV => RELU => POOL
    model.add(layers.Convolution2D(20,(5,5),activation='relu',input_shape=input_shape))
    model.add(layers.MaxPooling2D(pool_size=(2,2),strides=(2,2)))
    # CONV => RELU => POOL
    model.add(layers.Convolution2D(50,(5,5),activation='relu',input_shape=input_shape))
    model.add(layers.MaxPooling2D(pool_size=(2,2),strides=(2,2)))
    # Flatten => RELU layers
    model.add(layers.Flatten())
    model.add(layers.Dense(500,activation='relu'))
    # a softmax classifier
    model.add(layers.Dense(classes,activation="softmax"))
    return model

In [6]:
(X_train,y_train),(X_test,y_test) = datasets.mnist.load_data()
# reshape
X_train = X_train.reshape((60000,28,28,1))
X_test = X_test.reshape((10000,28,28,1))
# normalize
X_train, X_test = X_train / 255.0,X_test / 255.0
# cast
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
# convert class vectors to binary class matrices
y_train = tf.keras.utils.to_categorical(y_train,NB_CLASSES)
y_test = tf.keras.utils.to_categorical(y_test,NB_CLASSES)
# initialize the optimizer and model
model = build(input_shape=INPUT_SHAPE,classes=NB_CLASSES)
model.compile(loss="categorical_crossentropy",optimizer=OPTIMIZER,metrics=["accuracy"])
model.summary()


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 24, 24, 20)     │           520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 12, 12, 20)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 8, 8, 50)       │        25,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 4, 4, 50)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 800)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 500)            │       400,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         5,010 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 431,080 (1.64 MB)

 Trainable params: 431,080 (1.64 MB)

 Non-trainable params: 0 (0.00 B)

In [7]:
# use TensorBoard
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
# fit
history = model.fit(X_train,y_train,batch_size=BATCH_SIZE,epochs=EPOCHS,verbose=VERBOSE,
                    validation_split=VALIDATION_SPLIT,callbacks=tensorboard_callback)
score = model.evaluate(X_test,y_test,verbose=VERBOSE)
print("\nTest score:",score[0])
print("Test accuracy:",score[1])

Epoch 1/5
16/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4174 - loss: 1.9400 

I0000 00:00:1711268247.365765     122 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1711268247.382800     122 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - accuracy: 0.4941 - loss: 1.7262

W0000 00:00:1711268249.345211     125 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1711268250.540790     122 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1711268252.185966     122 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


24/24 ━━━━━━━━━━━━━━━━━━━━ 10s 216ms/step - accuracy: 0.5014 - loss: 1.7042 - val_accuracy: 0.7966 - val_loss: 0.5909
Epoch 2/5
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - accuracy: 0.8651 - loss: 0.4224 - val_accuracy: 0.9068 - val_loss: 0.3114
Epoch 3/5
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - accuracy: 0.9328 - loss: 0.2378 - val_accuracy: 0.9302 - val_loss: 0.2281
Epoch 4/5
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - accuracy: 0.9526 - loss: 0.1736 - val_accuracy: 0.9390 - val_loss: 0.1954
Epoch 5/5
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - accuracy: 0.9619 - loss: 0.1310 - val_accuracy: 0.9326 - val_loss: 0.2178
 88/313 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9359 - loss: 0.2096

W0000 00:00:1711268257.266719     125 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9311 - loss: 0.2195

Test score: 0.19435028731822968
Test accuracy: 0.9394999742507935


W0000 00:00:1711268258.426965     124 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
